# Пайплайн Baseline подхода с GigaChat

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#root_path = "/content/drive/MyDrive/Diploma-mag"

In [ ]:
# pip install -q -U langchain langchain-gigachat

In [ ]:
from langchain_gigachat import Gigachat

def init_gigachat():
    return Gigachat(credentials="ваш_ключ_авторизации", model="GigaChat-Max", verify_ssl_certs=False, temperature=1e-15, timeout=100)

llm = init_gigachat()

In [ ]:
question = "Как установить PhotoShop?"
response = llm.invoke(question)

# Пайплайн Baseline подхода LLaVA

In [ ]:
#pip install -q -U transformers bitsandbytes accelerate torch

In [ ]:
from transformers import BitsAndBytesConfig, LlavaNextProcessor, LlavaNextForConditionalGeneration
from PIL import Image
import io
from typing import Tuple

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    low_cpu_mem_usage=True,
    use_flash_attention_2=True
)

def get_qa_prompt(model_id:str, system_prompt:str, question: str, context: str, image: Image=None) -> str:
    return  f"[INST]{'<image>' if image else ' '}\n{system_prompt}\n{context}\n\nQuestion:\n{question}\n\n[/INST]"


def format_output(raw_output, processor: LlavaNextProcessor, prompt: str) -> str:
    out = processor.decode(raw_output[0], skip_special_tokens=True)
    out_prompt = prompt.replace("<image>", " ").strip()
    formatted_output = out.replace(out_prompt, "").strip()
    return formatted_output


def get_prompt(task: str, model_id: str, system_prompt: str, text: str, image: Image, question: str) -> str:
    prompt = get_qa_prompt(model_id, system_prompt, question, text, image)
    return prompt


def llava_call(prompt: str, model: LlavaNextForConditionalGeneration, processor: LlavaNextProcessor, device: str, image: Image=None) -> str:
    inputs = processor(prompt, image, return_tensors="pt").to(device)
    raw_output = model.generate(**inputs, max_new_tokens=300)
    formatted_output = format_output(raw_output, processor, prompt)
    return formatted_output


def load_llava_model(model_id: str) -> Tuple[LlavaNextForConditionalGeneration, LlavaNextProcessor]:
    processor = LlavaNextProcessor.from_pretrained(model_id)
    model = LlavaNextForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto", cache_dir=root_path + "/models")
    #model = LlavaNextForConditionalGeneration.from_pretrained(model_id, device_map="auto")
    return model, processor

In [ ]:
device = "cuda"
question = "Как выделить прямоугольную область на изображении в Adobe Photoshop?"
model, processor = load_llava_model("llava-hf/llava-v1.6-mistral-7b-hf")
model = model.eval()

no_img_prompt = f"[INST]\nQuestion:\n{question}\n\n[/INST]"

response = llava_call(no_img_prompt, model, processor, device)

# Оценка ответа (DeepSeek-R1)

In [ ]:
from langchain_gigachat import Gigachat
def init_deepseek():
    return Gigachat(credentials="ваш_ключ_авторизации", model="DeepSeek-R1", verify_ssl_certs=False, temperature=1e-15, timeout=100)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from prompts import ANSWER_CORRECTNESS_SYSTEM, ANSWER_CORRECTNESS_USER

answer_correctness = ChatPromptTemplate.from_messages(
    [
        ("system", ANSWER_CORRECTNESS_SYSTEM),
        ("human", ANSWER_CORRECTNESS_USER)
    ]
)
llm = init_deepseek()

answer_correctness_chain = answer_correctness | llm
response = answer_correctness_chain.invoke("question": question, "reference_answer": reference_answer, "generated_answer": generated_answer)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from prompts import ANSWER_RELEVANCE_SYSTEM, ANSWER_RELEVANCE_USER

answer_relevance = ChatPromptTemplate.from_messages(
    [
        ("system", ANSWER_RELEVANCE_SYSTEM),
        ("human", ANSWER_RELEVANCE_USER)
    ]
)
llm = init_deepseek()

answer_relevance_chain = answer_relevance | llm
generated_answer = """Чтобы выделить прямоугольную область в Photoshop, используйте инструмент "Прямоугольная область" (Rectangular Marquee Tool) на панели инструментов"""
response = answer_relevance_chain.invoke("query": question, "text": generated_answer)